# Creando una pagina web en html usando la información ya procesada

In [ ]:
import pandas as pd
from jinja2 import Template
import plotly.express as px

# Leer el archivo CSV
df = pd.read_csv("C:/Users/Josue/Desktop/trabajofinal/TRABAJO_FINAL_LP2/productos_segmentados.csv")

# Seleccionar las columnas nutricionales relevantes y la columna de Segmento
columnas_nutricionales = ["Energía (kcal)", 'Grasas (g)', 'Hidratos de Carbono (g)', 'Proteínas (g)', 'Azúcares (g)']  # Ajusta según las columnas de tu archivo CSV
segmento = 'Segmento'

# Calcular los promedios nutricionales agrupados por Segmento y redondear a 3 decimales
promedios_agrupados = df.groupby(segmento)[columnas_nutricionales].mean().reset_index()
promedios_agrupados = promedios_agrupados.round(3)

# Calcular la cantidad de productos por Segmento
cantidad_por_segmento = df[segmento].value_counts().reset_index()
cantidad_por_segmento.columns = [segmento, 'Cantidad']

# Generar el gráfico de barras para cantidad por segmento
fig_cantidad_por_segmento = px.bar(cantidad_por_segmento, x=segmento, y='Cantidad', title='Cantidad de Productos por Segmento')

# Calcular la matriz de correlación
columnas_correlacion = ['Energía (kcal)', 'Grasas (g)', 'Proteínas (g)', 'Hidratos de Carbono (g)', 'Azúcares (g)']
correlation_matrix = df[columnas_correlacion].corr().round(3)

# Crear el heatmap de la matriz de correlación
fig_correlation_matrix = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.values,
    colorscale='Viridis'
)
                 
# Obtener los 5 productos con más y menos Energía (kcal)
top_5_mas_energia = df.nlargest(5, 'Energía (kcal)')[['Producto', 'Energía (kcal)']]
top_5_menos_energia = df.nsmallest(5, 'Energía (kcal)')[['Producto', 'Energía (kcal)']]

# Análisis de Ingredientes
df['Número de Ingredientes'] = df['Ingredientes'].apply(lambda x: len(str(x).split(',')))
producto_Alta_Energía = df.loc[(df['Segmento']=="Alta Energía"), ['Producto', 'Número de Ingredientes', 'Ingredientes']]
producto_Bajo_en_Grasa = df.loc[(df['Segmento']=="Bajo en Grasa"), ['Producto', 'Número de Ingredientes', 'Ingredientes']]
producto_Otro = df.loc[(df['Segmento']=="Otro"), ['Producto', 'Número de Ingredientes', 'Ingredientes']]
producto_Alta_Proteína = df.loc[(df['Segmento']=="Alta Proteína"), ['Producto', 'Número de Ingredientes', 'Ingredientes']]


# Generar HTML con la tabla de promedios nutricionales agrupados por Segmento, el gráfico de barras, las tablas adicionales, la tabla de preferencias de mercado y la tabla de análisis de ingredientes
template = Template('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Promedios Nutricionales de OpenFoodFacts por Segmento</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f5f5dc; /* Beige background */
            color: #333333;
            margin: 0;
            padding: 20px;
        }
        h1, h2 {
            color: #8b4513; /* SaddleBrown */
            text-align: center;
        }
        table {
            border-collapse: collapse;
            width: 80%;
            margin: 20px auto;
            background-color: #ffffff; /* White background */
        }
        table th, table td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        table th {
            background-color: #f4a460; /* SandyBrown */
            color: white;
        }
        table tr:nth-child(even) {
            background-color: #f2f2f2; /* Light grey */
        }
        table tr:hover {
            background-color: #ddd; /* Darker grey */
        }
        .chart-container {
            width: 80%;
            margin: 0 auto;
            padding: 20px;
        }
        .comentario {
            text-align: center;
            font-style: italic;
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <h1>Promedios Nutricionales de OpenFoodFacts por Segmento</h1>
    <table>
        <thead>
            <tr>
                <th>{{ segmento }}</th>
                {% for col in columnas_nutricionales %}
                <th>{{ col }}</th>
                {% endfor %}
            </tr>
        </thead>
        <tbody>
            {% for index, row in promedios_agrupados.iterrows() %}
            <tr>
                <td>{{ row[segmento] }}</td>
                {% for col in columnas_nutricionales %}
                <td>{{ row[col] }}</td>
                {% endfor %}
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <div class="chart-container">
        {{ fig_html|safe }}
    </div>
    <h2>Top 5 Productos con Más Energía (kcal)</h2>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Energía (kcal)</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in top_5_mas_energia.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Energía (kcal)'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <h2>Top 5 Productos con Menos Energía (kcal)</h2>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Energía (kcal)</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in top_5_menos_energia.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Energía (kcal)'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <h2>Preferencias de Mercado</h2>
    <table>
        <thead>
            <tr>
                <th>{{ segmento }}</th>
                <th>Cantidad</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in cantidad_por_segmento.iterrows() %}
            <tr>
                <td>{{ row[segmento] }}</td>
                <td>{{ row['Cantidad'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <div class="chart-container">
        {{ fig_cantidad_por_segmento|safe }}
    </div>
    <div class="comentario">
        Esto nos indica que más hay una demanda en los productos del segmento de Alta Energía seguido de los productos Bajos en Grasa, lo que significa que los productos de estos dos segmentos tienen una mayor oferta o una mayor diversidad dentro de esas categorías.
    </div>
    <h2>Análisis de Correlación</h2>
    <div id="correlation_matrix">{{ fig_correlation_matrix|safe }}</div>
    <div class="comentario">
        La matriz de correlación nos muestra que la fuerte correlación entre Energía y Grasas sugiere que los alimentos altos en grasas tienden a ser más calóricos. 
        La correlación negativa entre Proteínas y Azúcares podría indicar que los alimentos ricos en proteínas tienden a ser bajos en azúcares, y viceversa. 
        La correlación positiva entre Hidratos de Carbono y Azúcares muestran una correlación positiva con Azúcares, lo cual es lógico ya que los azúcares son un tipo de carbohidrato.
    </div>
    <h2>Análisis de Ingredientes</h2>
    <h3>Productos Altos en Energía</h3>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Número de Ingredientes</th>
                <th>Ingredientes</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in producto_Alta_Energía.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Número de Ingredientes'] }}</td>
                <td>{{ row['Ingredientes'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <h3>Productos Bajos en Grasa</h3>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Número de Ingredientes</th>
                <th>Ingredientes</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in producto_Bajo_en_Grasa.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Número de Ingredientes'] }}</td>
                <td>{{ row['Ingredientes'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <h3>Otros Productos</h3>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Número de Ingredientes</th>
                <th>Ingredientes</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in producto_Otro.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Número de Ingredientes'] }}</td>
                <td>{{ row['Ingredientes'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <h3>Productos Altos en Proteína</h3>
    <table>
        <thead>
            <tr>
                <th>Producto</th>
                <th>Número de Ingredientes</th>
                <th>Ingredientes</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in producto_Alta_Proteína.iterrows() %}
            <tr>
                <td>{{ row['Producto'] }}</td>
                <td>{{ row['Número de Ingredientes'] }}</td>
                <td>{{ row['Ingredientes'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    <div class="comentario">
        Como se puede observar en esta comparación de ingredientes, los productos altos en proteína tienen menos cantidad de ingredientes, lo que indica que son productos más naturales o menos procesados en comparación con los productos bajos en grasas que tienen mayor cantidad de ingredientes en la mayoría de los productos.
    </div>
</body>
</html>
''')


# Renderizar el HTML con los promedios agrupados, el gráfico de barras, las tablas adicionales, la tabla de preferencias de mercado y la tabla de análisis de ingredientes
html_content = template.render(
    segmento=segmento,
    columnas_nutricionales=columnas_nutricionales,
    promedios_agrupados=promedios_agrupados,
    top_5_mas_energia=top_5_mas_energia,
    top_5_menos_energia=top_5_menos_energia,
    cantidad_por_segmento=cantidad_por_segmento,
    correlation_matrix=correlation_matrix,
    producto_Alta_Energía=producto_Alta_Energía,
    producto_Bajo_en_Grasa=producto_Bajo_en_Grasa,
    producto_Otro=producto_Otro,
    producto_Alta_Proteína=producto_Alta_Proteína,
    fig_cantidad_por_segmento=fig_cantidad_por_segmento.to_html(full_html=False, include_plotlyjs='cdn'),
    fig_correlation_matrix=fig_correlation_matrix.to_html(full_html=False, include_plotlyjs='cdn')
)

# Guardar el HTML en un archivo
with open('Trabajo.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("Archivo HTML con promedios nutricionales agrupados por Segmento, gráficos de barras y tablas adicionales generado exitosamente.")
